
# Data Wrangle OpenStreetMaps Data
#### Mahlon Barrault
#### August 28, 2015
#### Map Area: Calgary, Alberta (Map Zen Extract (https://mapzen.com/data/metro-extracts) includes suburbs) 

## Table of Contents

[Problems Encountered in the Map](#Problems-Encountered-in-the-Map)
    
* [Directional Suffixes and Street Names](#Directional-Suffixes-and-Street-Names)
* [Postal Codes](#Postal-Codes)
* [Rural Roads](#Rural-Roads)

[Data Overview](#Data-Overview)

[Additional Ideas](#Additional-Ideas)

[Conclusion](#Conclusion)

For this project I chose to use the map data from the area in which I live, Calgary, Alberta, Canada. Being a local I have an advantage in domain knowledge. Leveraging information I have acquired from living here I was able to produce more accurate standardizations. One example, which you will read more about later, was directional suffixes. In the data they appeared in a wide range of formats, however on signage in Calgary a direction initialism is used without punctuation. I invite you to read more about my process in the following section.

### Problems Encountered in the Map

From the outset of this project I was interested in loading as much data into MongoDB as was possible from the MapZen extract of Calgary. My motivation for this was to not stiffle any analyical possiblities later in the process because of not having the right data at hand.

The tag element in the extract file contained k and v attributes that I wanted to unwind and import in to MongoDB. I used the following regular expression to find any keys with problematic characters.

In [9]:
import re

# This regex represents invalid MongoDB characters for keys
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

No keys with problem characters were found. Next I wanted to see what sort of tags and attribute I would be dealing with.

In [11]:
%run mapparser.py

Most of these attributes were expected. The 'type' keys would have conflicted with the 'type' key that was going to be used to represent the tag names. I took note of this so later the key that would have the tag name could be renamed to 'node_type'. 

#### Directional Suffixes and Street Types

Now that I was sufficiently confident that I would be able to translate the tag and attributes from the extract file into well formed MongoDB documents, it was time to turn my attention to the deeper contents in the attributes.

An ideally formatted Calgary address looks like 1234 StreetName StreetType DirectionSuffix. I wrote the following helper functions to help me disassemble the 'addr:street' key and get a sense for what other problems might exist.

In [12]:
def audit_street_type(street_types, street_name):
    ''' Modfies street_types dict. Adds street_name to dict if it does not have
    a conformed street type or conformed directional suffix.
    '''
    m = get_suffix(street_name)
    if m:
        street_type = m.group()
        if street_type not in ST_TYPE_EXPECTED:
                street_types[street_type].add(street_name)

def get_suffix(street_name):
    ''' Returns the last word from street_name with any directional suffixes
    removed.
    ''' 
    return street_type_re.search(street_type_re.sub(remove_dir, street_name))

def remove_dir(m):
    ''' Accepts a RegEx match. Returns and empty string if the match is an
    expected directional suffix. Returns the whole match if it is not a 
    directional suffix.
    '''
    if m.group().upper() in DIR_EXPECTED:
        return ''
    else:
        return m.group()
    return

def process_match(m):
    ''' Returns the value from ST_TYPE_MAPPING with corresponding key from RegEx
    search. If no key exists the orginal search results from m are returned 
    unmodified.
    '''
    if ST_TYPE_MAPPING.get(m.group()) != None:
        return ST_TYPE_MAPPING.get(m.group())
    else:
        return m.group()
    return 


def update_name(name):
    ''' Returns conformed name if there was a mapping in ST_TYPE_MAPPING.
    Otherwise returns name.
    '''
    return street_type_re.sub(process_match, name)

I found several addresses that were not ideally formatted. Here is a sample of some of the problems that were found.

In [13]:
sample_problems = {'Northeast': set(['16 Avenue Northeast']),'West': set(['Pine Ridge Road West', '206 Avenue West']), \
 'Cres': set(['Bracewood Cres']), 'N.W': set(['Symons Valley Rd N.W']), 'South-west': set(['Everhollow Manor  South-west']), \
 'Alberta': set(['Veterans Blvd., Airdrie, Alberta']), 'AB': set(['32nd Street, Okotoks, AB']), \
 'Blvd.': set(['Vetrans Blvd.']), 'street': set(['2 street'])}

For the directional suffixes I saw that the full words were used as in 'Northeast'. Sometimes they were hyphenated as with 'South-west' or they were an initialism but with punctuation separators like 'N.W'. With the street types there were abbreviations, abbreviations with punctuation. Sometimes the province had been included in the address, so that would need to be accommodated. The last example in the dictionary is an example of a street without a directional suffix.

Armed with a exhaustive list of all of the directional suffix and street name problems I could go to work cleaning them. At the heart of the cleaning operation was the following recursive (and well documented) function. I choose to over-comment this function because recursion can bend your mind sometimes.

In [14]:
def update_st_name(street_name):
    ''' Recursive function to conform street types and directional suffixes'''
    street_name = street_name.split(',')[0]

    init_search = street_type_re.search(street_name)
    
    if init_search:
        init_search = init_search.group()
        
        if init_search in DIR_EXPECTED:
            # street_name should now have the street type exposed on the end
            # of the string
            street_name = street_type_re.sub(remove_suffix, street_name)
            # Recusive call to have the funtion check street type. strip() is
            # needed so that the RegEx matches properly
            return str(update_st_name(street_name.strip())) + ' ' + init_search
        elif init_search in DIR_MAPPING:
            dir_clean = DIR_MAPPING[init_search]
            street_name = street_type_re.sub(remove_suffix, street_name)
            # Recusive call to have the funtion check street type. strip() is
            # needed so that the RegEx matches properly
            return str(update_st_name(street_name.strip())) + ' ' + dir_clean
        elif init_search in ST_TYPE_EXPECTED:
            # Recursive base case
            return street_name
        elif init_search in ST_TYPE_MAPPING:
            st_ty_clean = ST_TYPE_MAPPING[init_search]
            street_name = street_type_re.sub(st_ty_clean, street_name)
            # Alternate recursive base case
            return street_name
        else:
            # Catches streets that will not be cleaned like 
            # 'Township Road  204A'
            return street_name
    else:
        # Recursion should not hit this else, something has gone wrong; will
        # return 'None'
        return street_name

The job of this function was to take apart the incoming address and reassemble it with a conformed directional suffix and street type.

In [18]:
update_st_name('16 Avenue Northeast')
'16 Avenue NE'
update_st_name('Pine Ridge Road West')
'Pine Ridge Road W'
update_st_name('Bracewood Cres')
'Bracewood Crescent'
update_st_name('Symons Valley Rd N.W')
'Symons Valley Road NW'
update_st_name('Everhollow Manor  South-west')#
'Everhollow Manor SW'

From the basic tests above we can see update_st_name() is doing a great job. 

#### Postal Codes

In Canada there is wide spread confusion regarding how postal codes are to be written. According to Canada Post's guidelines: "The Postal CodeOM should be printed in upper case with the first three characters separated from the last three by one space. Do not use hyphens." [<sup>1</sup>](#References). This will be the standard I will conform postal codes in the data set to.

Altogether there were 49 improperly formatted codes 33 of which were unique. Initially I wanted to clean these values by simply creating a defaultdict map between the malformed keys and the set of manually conformed keys. Despite being a manageable number of changes to handle manually, creating defaultdicts by hand is not a particularly scalable technique. Instead I created the following function to programmatically generate the defaultdict which could be used to update a collection in MongoDB.

In [19]:
def clean_post_codes(db):
    '''
    Accepts a mongodb instance, finds and cleans malformed postal codes in the
    provided db. Assumes the database has a collection called DANDP3.
    '''
    post_code_changes = defaultdict(set)
    bad_post_codes = []

    # Finds strings that have the correct letter number alternation but might not
    # have the correct segment separator
    re_valid_post_code = re.compile('[A-Za-z]\d[A-Za-z]( ?\-?)\d[A-Za-z]\d')
    
    # Retreive nonconformed post codes from mongodb
    bad_postcode_docs = db.DANDP3.find({'address.postcode' : {'$exists': 'true', \
    '$not' : re.compile('^([A-Z]\d[A-Z]( )\d[A-Z]\d)')}})
    
    for d in bad_postcode_docs:
        bad_post_codes.append(d)
        
    for bpc in bad_post_codes:
        match = re_valid_post_code.search(bpc)
        
        if match:
            post_code = match.group()
            post_code = post_code.upper()
            
            if post_code[3] != ' ':
                if len(post_code) <= 6:
                    # Insert a space if it is missing
                    post_code = post_code[0:3] + ' ' + post_code[3:6]
                elif len(post_code) > 6:
                    # post code len is good but char 3 is not a space, replace it
                    post_code = post_code.replace(post_code[3], ' ')
            post_code_changes[bpc] = post_code
        else:
            post_code_changes[bpc] = ''

clean_post_codes uses two slightly different regular expressions: one to find all postal code-like values and another to find which of them are malformed. The rest of the function used string slices and built-in string methods to manipulate the postal code in to submission. Any values that bear little to no resemblance to a postal code are set to an empty string. After running this function, the defaultdict post_code_changes looks like this:

```
defaultdict(set,
            {' T2J 0P8': 'T2J 0P8',
             '1212': '',
             '403-719-6250': '',
             'AB T2G 2L2': 'T2G 2L2',
             'AB T2S 2N1': 'T2S 2N1',
             'T1X1L8': 'T1X 1L8',
             'T2E': '',
             'T2G0H7': 'T2G 0H7',
             'T2J2T8': 'T2J 2T8',
             'T2L1G1': 'T2L 1G1',
             'T2P0W3': 'T2P 0W3',
             'T2P3P8': 'T2P 3P8',
             'T2R0E7': 'T2R 0E7',
             'T2T0A7;T2T 0A7': 'T2T 0A7',
             'T2V2X3': 'T2V 2X3',
             'T3A0H7': 'T3A 0H7',
             'T3A5R8': 'T3A 5R8',
             'T3A6J1': 'T3A 6J1',
             'T3B3X3': 'T3B 3X3',
             'T3G2V7': 'T3G 2V7',
             'T3J0G7': 'T3J 0G7',
             'T3J0S3': 'T3J 0S3',
             'T3J4L8': 'T3J 4L8',
             'T3K-5P4': 'T3K 5P4',
             'T3N0A6': 'T3N 0A6',
             'T3N0E4': 'T3N 0E4',
             'T3R0A1': 'T3R 0A1',
             'T3R0H3': 'T3R 0H3',
             'T3a4b3': 'T3A 4B3',
             't2n 3P3': 'T2N 3P3',
             't2n 4l7': 'T2N 4L7',
             't3G 5T3': 'T3G 5T3',
             't3c2h6': 'T3C 2H6'})
```

In the examples above we can see that the function is handling the incoming values appropriately and we are left with a clean conformed set of postal codes! This dictionary can now be used to update MongoDB or do something else like get additional metadata about the postal code from another database or API.

#### Rural Roads

This dataset encompassed a very large area and included large parts of farm area around the City of Calgary. As a result there are several rural roads like 'Township Road 204A' which aren't strictly part of the city. Their inclusion did not cause significant issues but they do not add much value to the dataset.

### Data Overview

##### File Sizes
calgary_canada.osm : 159 MB

calgary_canada.osm.json : 184 MB

##### Number of Documents
    db.DANDP3.count() : 863038

###### See analyze.py for full code for the following

##### Largest Document
    print get_largest_doc(get_all_docs(db)) : 112886 characters, 'Proposed West Stoney Trail'

##### Number of Unique Users
    print len(get_users(docs)) : 767

##### Top Three Contributors
    print users[0:3] : [{u'count': 309818, u'_id': u'sbrown'}, {u'count': 89769, u'_id': u'Zippanova'}, {u'count': 46296, u'_id': u'markbegbie'}]

##### Rank of My Contributions
    print 'My Rank ' + mb_rank : 157

##### Number of Ways
    print 'Number of Ways: ' + str(db.DANDP3.find({"node_type":"way"}).count()) : 83492

##### Number of Nodes
    print 'Number of Nodes: ' + str(db.DANDP3.find({"node_type":"node"}).count()) : 779009

### Additional Ideas

#### Postal Code Standarization
During my post import cleaning I made several standardization corrections to the postcodes in the dataset. However, even the correctly formatted postcodes may not be legitimate postcodes. I thought that a next step would be to connect to Canada Post's address validation API and confirm that the postal codes exist. This would allow me to collect some additional metadata regarding what are each valid postal code covers. This could be compared to the coordinates of the object from the map data and further validate the postal code and possibly suggest a correct postal code based on the coordinates.

I would have to develop functions that could communicate with the Canada Post address validation API. This would allow me to fully validate whole addresses including the postcodes and then take corrective action. The Canada Post API has the capability to suggest addresses that are closely matching the query submitted. 

From examining the Canada Post API it seems that it is designed to validate whole addresses and not necessarily the components of the address. To utilize this API additional effort into auditing and cleaning the other address components would be needed. Many of the nodes in the dataset are for ambiguous areas or objects. Some of these may not have real world physical addresses so full address conformation may not be possible for many nodes. From the information supplied on some nodes there may be several matching addresses for that node. A strategy would have to be implemented for picking the most appropriate address for each node.

This might pose an interesting challenge to do this autonomously without user intervention. 

#### Analyze User Contributions by Area

Another potential analysis might be to examine what areas of the city users tend to make contributions. Are there patterns in user contribution that would indicate where that users lives or works? Are there more contributions in newer areas of the city or is it evenly distributed? How much does the City of Calgary contribute? These are just a few questions that I would be interested in digging into further. Data visualization could be used to get a basic idea of clusters of contribution activity. 

To get the most satisfying answers machine learning would have to be implemented to expose these correlations in the data. It would be challenging to accurately isolate each area of the city to analyze the user contribution activity. Data from the city's website would need to be used to enrich the OpenStreetMaps data including neighborhood boundaries, city wards and demographics. These data would likely need a similar munging treatment to be usable.

##### Additional Analysis

##### How many places in Calgary accept bitcoin?
    len(list(db.DANDP3.find({"payment:bitcoin" : "yes"}))) : 18 

##### What are the most common amenties?
    amenites = list(aggregate(db, [{'$match' : {'amenity' : {'$exists' : 1}}},\
                               {'$group' : {'_id' : '$amenity', 'count' : \
                               {'$sum' : 1}}}, {'$sort' : {'count' : -1}}]))
    print amenites[0:3]
    [{u'count': 1735, u'_id': u'parking'}, {u'count': 461, u'_id': u'restaurant'}, {u'count': 441, u'_id': u'fast_food'}]

##### What are the most common fast food places?

    fast_food_by_name = list(aggregate(db, [{'$match' : {'amenity' : {'$exists' : 1}, 'name' : {'$exists' : 1} }}, {'$group' : {'_id' : '$name', 'count' : {'$sum' : 1}}},{'$sort' : {'count' : -1}}]))
    print fast_food_by_name[0:3]
    
     [{u'_id': u'Tim Hortons', u'count': 59}, {u'_id': u'Subway', u'count': 54}, {u'_id': u'Shell', u'count': 36}]
     
##### Additional additonal analysis can be found in analyze.py starting at line 111.

### Conclusion

When working on the Additional Analysis section I discovered that there were some keys that I would have liked to take a closer look but their structure (key:sub_key) made it difficult to get good grouping. This solidified the lesson of audit, clean, repeat for me. Because of the nature of this issue I would have had to go back to the original file processing code and add logic to turn the key:sub_key entries to key : [sub_key : value] entries. Despite my efforts in cleaning the dataset there is still much more that can be done to improve it. 

Based on the vast number of contributors, it would seem that people are interested in keeping the map data for Calgary up-to-date. This has advantages and disadvantages for the quality of the data. The more users that contribute the more ways data could be entered incorrectly. 

Understanding that OpenStreetMaps endeavors to be open, it might be advantageous to intelligently offer suggestions as someone is entering data. This might reduce the number of ill-conceived keys in the data and improve the analysis of map data. It has been shown in many other open systems that intelligent suggestions are not always accepted or appreciated. Perhaps there should be some penalty for creating new keys like having to submit a request to a moderator to have the key added to an object. This might influence contributor behavior to accept suggestions or reconsider if a new key is really necessary. However, this could also have unintended consequences. User frustration could increase and user contributions could suffer. Users might begin to store information on keys where it would not be meaningful or relevant to do so.

While working with OpenStreetMaps data I reflected on what an undertaking it would be to consume these data to enrich a service like Google Maps. A pipeline to ingest these data would have to account for many of the factors that I had to consider plus many more. I have only examined a small fraction of the OpenStreeMaps data and there was a wide range of data quality challenges. I would be interested to see if the techniques I employed are similar to engineers who work on systems that consider ALL OpenStreetMaps data. No doubt, as we produce more and more data as a society we will need to develop systems that assist the data wrangler. For now, we will just have to keep auditing, cleaning, and repeating.

### References

1. Canada Post. (2016). Addressing Guidelines | Canada Post. [online] Canadapost.ca. Available at: https://www.canadapost.ca/tools/pg/manual/PGaddress-e.asp?ecid=murl10006450 [Accessed 19 Nov. 2016].